In [ ]:
import os 
import json

def dirlist(path, allfile=[], recursion=True):
    """
    递归列出路径下文件
    :param path:
    :param allfile:
    :param recursion:
    :return:
    """
    filelist = os.listdir(path)
    for filename in filelist:
        filepath = os.path.join(path, filename)
        filepath = os.path.abspath(filepath)
        if os.path.isdir(filepath):
            if recursion:
                dirlist(filepath, allfile)
        else:
            allfile.append(filepath)
    return allfile



files = dirlist('res/', [])
files[0]


In [ ]:
import json
import numpy as np

# 获取数据
id,insert_time,search_time,all_time = [], [], [], []

for file in files:
    with open(file, 'r', encoding='utf-8') as op:
        src = op.readlines()
        for line in src:
          obj = json.loads(line.strip())
          id.append(obj['id'])
          insert_time.append(obj['insert_time'])
          search_time.append(obj['search_time'])
          all_time.append(obj['insert_time']+obj['search_time'])

len(id)

In [ ]:
# 查看分位数
percentile_insert = np.percentile(insert_time,[50,90,99,99.9,99.99,100])
percentile_search = np.percentile(search_time,[50,90,99,99.9,99.99,100])
percentile_all = np.percentile(all_time,[50,90,99,99.9,99.99,100])
percentile_insert,percentile_search,percentile_all

In [ ]:
import pyecharts.options as opts
from pyecharts.charts import Line
# 每500个数值 取一次平均
each = 500
batch = len(id) // each if len(id) % each == 0 else len(id) // each + 1
print(batch)

# 平均时间
batch_id_mean, batch_insert_time_mean, batch_search_time_mean,batch_all_time_mean = [],[],[],[]

for i in range(batch):
  batch_id_mean.append(i)
  batch_insert_time_mean.append(sum(insert_time[i*each:(i+1)*each])/each)
  batch_search_time_mean.append(sum(search_time[i*each:(i+1)*each])/each)
  batch_all_time_mean.append(sum(all_time[i*each: (i+1)*each])/each)

# p99
batch_id_p99, batch_insert_time_p99, batch_search_time_p99,batch_all_time_p99 = [],[],[],[]

for i in range(batch):
  batch_id_p99.append(i)
#   print(i, np.percentile(insert_time[i*each:(i+1)*each],[99]))
  batch_insert_time_p99.append(np.percentile(insert_time[i*each:(i+1)*each],[99])[0])
  batch_search_time_p99.append(np.percentile(search_time[i*each:(i+1)*each],[99])[0])
  batch_all_time_p99.append(np.percentile(all_time[i*each:(i+1)*each],[99])[0])

# p999
batch_id_p999, batch_insert_time_p999, batch_search_time_p999,batch_all_time_p999 = [],[],[],[]

for i in range(batch):
  batch_id_p999.append(i)
  batch_insert_time_p999.append(np.percentile(insert_time[i*each:(i+1)*each],[99.9])[0])
  batch_search_time_p999.append(np.percentile(search_time[i*each:(i+1)*each],[99.9])[0])
  batch_all_time_p999.append(np.percentile(all_time[i*each:(i+1)*each],[99.9])[0])

In [ ]:
# 平均时间

In [ ]:
line=(
    Line()
    .add_xaxis(xaxis_data=batch_id_mean)
    .add_yaxis(series_name="insert_time",y_axis=batch_insert_time_mean,is_symbol_show=False)
    .add_yaxis(series_name="search_time",y_axis=batch_search_time_mean,is_symbol_show=False)
    .add_yaxis(series_name="all_time",y_axis=batch_all_time_mean,is_symbol_show=False)
    .set_global_opts(title_opts=opts.TitleOpts(title="milvus_latency_mean",pos_left="center"),
                    xaxis_opts=opts.AxisOpts(name='batch id',name_location="center",name_gap=40,interval=10),
                    yaxis_opts=opts.AxisOpts(name='latency(ms)',name_location="center",name_gap=40),
                    legend_opts=opts.LegendOpts(is_show=True,orient='vertical',pos_right=-5, pos_top=60)
)
)
line.render('batch_line_chart_mean.html')
line.render_notebook()

In [ ]:
# p99

In [ ]:
line=(
    Line()
    .add_xaxis(xaxis_data=batch_id_p99)
    .add_yaxis(series_name="insert_time",y_axis=batch_insert_time_p99,is_symbol_show=False)
    .add_yaxis(series_name="search_time",y_axis=batch_search_time_p99,is_symbol_show=False)
    .add_yaxis(series_name="all_time",y_axis=batch_all_time_p99,is_symbol_show=False)
    .set_global_opts(title_opts=opts.TitleOpts(title="milvus_latency_p99",pos_left="center"),
                    xaxis_opts=opts.AxisOpts(name='batch id',name_location="center",name_gap=40),
                    yaxis_opts=opts.AxisOpts(name='latency(ms)',name_location="center",name_gap=40),
                    legend_opts=opts.LegendOpts(is_show=True,orient='vertical',pos_right=-5, pos_top=60)
)
)
line.render('batch_line_chart_p99.html')
line.render_notebook()

In [ ]:
# p999

In [ ]:

line=(
    Line()
    .add_xaxis(xaxis_data=batch_id_p999)
    .add_yaxis(series_name="insert_time",y_axis=batch_insert_time_p999,is_symbol_show=False)
    .add_yaxis(series_name="search_time",y_axis=batch_search_time_p999,is_symbol_show=False)
    .add_yaxis(series_name="all_time",y_axis=batch_all_time_p999,is_symbol_show=False)
    .set_global_opts(title_opts=opts.TitleOpts(title="milvus_latency_p999",pos_left="center"),
                    xaxis_opts=opts.AxisOpts(name='batch id',name_location="center",name_gap=40,min_interval=50),
                    yaxis_opts=opts.AxisOpts(name='latency(ms)',name_location="center",name_gap=40),
                    legend_opts=opts.LegendOpts(is_show=True,orient='vertical',pos_right=-5, pos_top=60)
)
)
line.render('batch_line_p999.html')
line.render_notebook()